# SDialog dependencies

In [1]:
%%script false --no-raise-error
# Only run this cell if you are using a non jupyter environment
!conda create --name jsalt python=3.9 -y
!conda activate jsalt
!conda install sox
!pip install -r sdialog/requirements.txt
!pip install -r sdialog/requirements-audio.txt

In [2]:
import os
import json
from tqdm import tqdm

In [3]:
import sdialog
from sdialog import Dialog
from sdialog.generators import PersonaGenerator
from sdialog.personas import Persona, PersonaAgent, Doctor, Patient, Agent

/Users/yanislabrak/opt/miniconda3/envs/jsalt/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Generate Persona

In [4]:
sdialog.config.set_llm("aws:anthropic.claude-3-5-sonnet-20240620-v1:0", region_name="us-east-1")

In [5]:
%%script false --no-raise-error
persona_card_folder = "./personas"

# Generate doctor persona
doctor_persona = Doctor(
    name="Dr. Smith",
    gender="male",
    age=52,
    specialty="Family Medicine"
)
generator_doctor = PersonaGenerator(doctor_persona)
persona_cards = generator_doctor.generate(n=1)
persona_cards.to_file(f"{persona_card_folder}/persona_doctor.json")

# Generate patient persona
patient_persona = Patient(
    name="John Doe",
    gender="male",
    age=62
)
generator_patient = PersonaGenerator(patient_persona)
persona_cards = generator_patient.generate(n=1)
persona_cards.to_file(f"{persona_card_folder}/persona_patient.json")


# Load persona

In [6]:
# Load personas
persona_doctor = Persona.from_file("./personas/persona_doctor.json")
persona_patient = Persona.from_file("./personas/persona_patient.json")

In [7]:
%%script false --no-raise-error
context = "Generate me a 50 turn medical dialogue between patient and doctor, for a primary care visit"

# Create agents
agent1 = PersonaAgent(persona=persona_doctor, name="DOCTOR", dialogue_details=context, response_details="make short turn answers when needed")
agent2 = PersonaAgent(persona=persona_patient, name="PATIENT", dialogue_details=context, response_details="make short turn answers when needed")

In [8]:
save_all = True
GENERATE_PERSONA = True
FORCE_DIALOG_GENERATION = False

os.makedirs("./outputs", exist_ok=True)

In [9]:
if FORCE_DIALOG_GENERATION:

    original_dialog = agent1.talk_with(agent2, max_turns=3)
    original_dialog.to_file("dialog_demo.json")

else:
    original_dialog = Dialog.from_file("dialog_demo.json")

original_dialog.print()

[dialog_id] 1752861530588
[model] client=<botocore.client.BedrockRuntime object at 0x7fc319ec91c0> model_id='anthropic.claude-3-5-sonnet-20240620-v1:0' region_name='us-east-1' provider='anthropic' supports_tool_choice_values=('auto', 'any', 'tool')
[seed] 226296126
--- Dialogue Begins ---
[DOCTOR] Hello there. I'm Dr. Smith. Welcome to my office. What brings you in to see me today?
[PATIENT] Good morning, Dr. Smith. I'm John Doe. I've been dealing with a persistent cough and feeling pretty tired for about three weeks now. It's starting to wear me down, and I thought I should get it checked out.
[DOCTOR] I'm sorry to hear you've been feeling unwell, Mr. Doe. A persistent cough and fatigue can certainly be troublesome. Let's get some more details. Can you describe your cough? Is it dry or productive?
[PATIENT] Well, Dr. Smith, it's mostly a dry cough. It's particularly bothersome at night when I'm trying to sleep. You know, it reminds me of the time I was teaching about the Civil War and

# Audio Generation

You can generate three type of audios:
- (default) Step 1: Raw utterances passed to a TTS model and concatenated to each others to create an audio file
- Step 2: Audio generated from multiple channels create using signal positions
- Step 3: Audio generated using room spacialization and multi-channels positions

If you want to trigger the "step 2" you need to give a Scaper argument to the `audio_pipeline`. While for the "step 3" you need also to give a "Room" in the `inference` function.

### Instanciate voices database

In [10]:
from sdialog.audio.voice_database import DummyVoiceDatabase
dummy_voice_database = DummyVoiceDatabase()

OR

In [ ]:
from sdialog.audio.voice_database import HuggingfaceVoiceDatabase
voices_libritts = HuggingfaceVoiceDatabase("sdialog/voices-libritts")

OR

In [ ]:
from sdialog.audio.voice_database import HuggingfaceVoiceDatabase
jsalt_voices = HuggingfaceVoiceDatabase("sdialog/voices-jsalt")

### Instanciate TTS model

In [11]:
from sdialog.audio.tts_engine import KokoroTTS
tts_pipeline = KokoroTTS()

/Users/yanislabrak/opt/miniconda3/envs/jsalt/lib/python3.9/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/Users/yanislabrak/opt/miniconda3/envs/jsalt/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


## Setup stage: Audio Dialog and Audio Pipeline

In [12]:
from sdialog.audio.audio_dialog import AudioDialog
from sdialog.audio.audio_pipeline import AudioPipeline

In [13]:
dialog: AudioDialog = AudioDialog.from_dialog(original_dialog)

## Step 1 : Concatenated utterances

In [14]:
audio_pipeline = AudioPipeline(
    voice_database=dummy_voice_database,
    tts_pipeline=tts_pipeline,
    dir_audio="./outputs",
)

OR

In [15]:
%%script false --no-raise-error
audio_pipeline = AudioPipeline() # Default values are used

In [16]:
dialog: AudioDialog = audio_pipeline.inference(dialog) # Generate the audio for the dialog
print(dialog.audio_step_1_filepath) # Path to the audio of the first stage of the audio pipeline

[2025-07-25 11:15:14] ERROR:asyncio:Future exception was never retrieved
future: <Future finished exception=BrokenPipeError(32, 'Broken pipe')>
Traceback (most recent call last):
  File "/Users/yanislabrak/opt/miniconda3/envs/jsalt/lib/python3.9/asyncio/unix_events.py", line 665, in write
    n = os.write(self._fileno, data)
BrokenPipeError: [Errno 32] Broken pipe
[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.
[2025-07-25 11:15:51] WARNING:root:The dSCAPER is not set, which make the generation of the timeline impossible
[2025-07-25 11:15:51] WARNING:root:The room or the dSCAPER is not set, which make the generation of the room accoustic audio impossible


./outputs/dialog_1752861530588/exported_audios/audio_pipeline_step1.wav


## Step 2: dScaper

In [ ]:
%%script false --no-raise-error
!git clone https://github.com/cyrta/dscaper.git

In [ ]:
%%script false --no-raise-error
%pip install -r ../../../requirements-dscaper.txt

In [17]:
import scaper
DATA_PATH = "./dscaper_data" # Path where the sound events, utterances and timelines database will be saved
os.makedirs(DATA_PATH, exist_ok=True)

In [18]:
dsc = scaper.Dscaper(dscaper_base_path=DATA_PATH)

In [20]:
audio_pipeline = AudioPipeline(dscaper=dsc)

/Users/yanislabrak/opt/miniconda3/envs/jsalt/lib/python3.9/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/Users/yanislabrak/opt/miniconda3/envs/jsalt/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [21]:
# Populate the sound events database
audio_pipeline.populate_dscaper(["sdialog/background","sdialog/foreground"])

Populating dSCAPER with sdialog/background dataset...:   0%|          | 0/4 [00:00<?, ?it/s][2025-07-25 11:16:47] ERROR:root:Problem storing audio /Users/yanislabrak/.cache/huggingface/hub/datasets--sdialog--background/snapshots/899f03337bf907855fa99ececdadbcabf3705ab6/ac_noise/351790__reiyamanor__small-room-tone-vintage-air-condition-at.wav (the audio can also be already stored)
Populating dSCAPER with sdialog/background dataset...:  25%|██▌       | 1/4 [00:01<00:05,  1.72s/it][2025-07-25 11:16:47] ERROR:root:Problem storing audio /Users/yanislabrak/.cache/huggingface/hub/datasets--sdialog--background/snapshots/899f03337bf907855fa99ececdadbcabf3705ab6/ac_noise/546572__tim_verberne__f_st_room_tone_04.wav (the audio can also be already stored)
[2025-07-25 11:16:47] ERROR:root:Problem storing audio /Users/yanislabrak/.cache/huggingface/hub/datasets--sdialog--background/snapshots/899f03337bf907855fa99ececdadbcabf3705ab6/fan_noise/210098__yuval__room-big-fan.wav (the audio can also be alre

0

In [22]:
dialog: AudioDialog = audio_pipeline.inference(dialog)
print(dialog.audio_step_1_filepath)
print(dialog.audio_step_2_filepath)

/Users/yanislabrak/Desktop/HUB/JSALT/sdialog/dscaper/scaper/core.py:1657: ScaperWarning: PATIENT event duration (18.40) is greater that source duration (18.38), changing to 18.38
  warnings.warn(
/Users/yanislabrak/Desktop/HUB/JSALT/sdialog/dscaper/scaper/core.py:1657: ScaperWarning: DOCTOR event duration (5.60) is greater that source duration (5.58), changing to 5.58
  warnings.warn(
/Users/yanislabrak/Desktop/HUB/JSALT/sdialog/dscaper/scaper/core.py:1657: ScaperWarning: DOCTOR event duration (12.40) is greater that source duration (12.38), changing to 12.38
  warnings.warn(
/Users/yanislabrak/Desktop/HUB/JSALT/sdialog/dscaper/scaper/core.py:1657: ScaperWarning: white_noise event duration (0.10) is greater that source duration (0.02), changing to 0.02
  warnings.warn(
[2025-07-25 11:17:34] INFO:root:Successfully generated dscaper timeline.
[2025-07-25 11:17:34] WARNING:root:The room or the dSCAPER is not set, which make the generation of the room accoustic audio impossible


./outputs/dialog_1752861530588/exported_audios/audio_pipeline_step1.wav
./outputs/dialog_1752861530588/exported_audios/audio_pipeline_step2.wav


## Step 3 : Room Accoustics

In [23]:
audio_pipeline = AudioPipeline(dscaper=dsc) # The audio pipeline doesn't change

/Users/yanislabrak/opt/miniconda3/envs/jsalt/lib/python3.9/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/Users/yanislabrak/opt/miniconda3/envs/jsalt/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [24]:
from sdialog.audio.room import MicrophonePosition
from sdialog.audio.room_generator import RoomGenerator, RoomRole

In [25]:
room = RoomGenerator().generate(RoomRole.CONSULTATION)
print(room)

5077:  RoomRole.CONSULTATION room_5077, desc: consultation room (dimentions: dim: [4.018706259482024, 2.363944858518838, 3.0], rt60: 0.5) role: RoomRole.CONSULTATION)  


In [26]:
dialog: AudioDialog = audio_pipeline.inference(
    dialog,
    room=room, # Need to provide a room object to trigger the 3rd step of the audio pipeline
    # microphone_position=MicrophonePosition.MONITOR # Default is MicrophonePosition.CEILING_CENTERED
)
print(dialog.audio_step_1_filepath)
print(dialog.audio_step_2_filepath)
print(dialog.audio_step_3_filepath)

[2025-07-25 11:18:36] ERROR:root:Problem storing audio for turn ./outputs/dialog_1752861530588/utterances/0_DOCTOR.wav
[2025-07-25 11:18:36] ERROR:root:Problem storing audio for turn ./outputs/dialog_1752861530588/utterances/1_PATIENT.wav
[2025-07-25 11:18:36] ERROR:root:Problem storing audio for turn ./outputs/dialog_1752861530588/utterances/2_DOCTOR.wav
[2025-07-25 11:18:36] ERROR:root:Problem storing audio for turn ./outputs/dialog_1752861530588/utterances/3_PATIENT.wav
/Users/yanislabrak/Desktop/HUB/JSALT/sdialog/dscaper/scaper/core.py:1739: ScaperWarning: ac_noise source time tuple (const, 0.0) could not be satisfied given source duration (30.00) and event duration (47.77), source time tuple changed to (const, 0) but was still not satisfiable, likely due to using 'normal' distribution with bounds too close to the start or end of the audio file
  warnings.warn(
/Users/yanislabrak/Desktop/HUB/JSALT/sdialog/dscaper/scaper/core.py:1657: ScaperWarning: DOCTOR event duration (12.40) is 

  Microphone set to position [1.1046765648705061, 0.3545917287778257, 1.2]
  Microphone set to position [2.009353129741012, 1.181972429259419, 2.9]
✓ Loaded audio file './dscaper_data/timelines/dialog_1752861530588/generate/fd8a0ff0-aa05-4fcf-ad23-9a84014d9450/soundscape_positions/no_type.wav' for 'no_type' with 2106877 samples
✓ Loaded audio file './dscaper_data/timelines/dialog_1752861530588/generate/fd8a0ff0-aa05-4fcf-ad23-9a84014d9450/soundscape_positions/doctor-at_desk_sitting.wav' for 'doctor-at_desk_sitting' with 2106877 samples
✓ Loaded audio file './dscaper_data/timelines/dialog_1752861530588/generate/fd8a0ff0-aa05-4fcf-ad23-9a84014d9450/soundscape_positions/patient-next_to_desk_sitting.wav' for 'patient-next_to_desk_sitting' with 2106877 samples
✓ Loaded audio file './dscaper_data/timelines/dialog_1752861530588/generate/2e539e6c-f23d-4b3f-8cc9-dc26c7f4d0b0/soundscape_positions/no_type.wav' for 'no_type' with 2106877 samples
✓ Loaded audio file './dscaper_data/timelines/dialog

## Automatic Position and SNR completion (under construction)

In [ ]:
from sdialog.audio.audio_events_enricher import AudioEventsEnricher
enricher = AudioEventsEnricher()

In [ ]:
audio_pipeline = AudioPipeline(enricher=enricher)

In [ ]:
for turn in dialog.turns:
    print(turn.text)
    print(turn.position)
    print(turn.snr)
    print("____________________")

## dialog.to_audio()

In [ ]:
# TODO: Add a demo of the audio generation pipeline from the dialog object

# Audio Evaluation

## Utterance level evaluation

In [ ]:
from sdialog.audio.evaluation import compute_evaluation_utterances, compute_evaluation_audio

In [ ]:
# Utterances level evaluation
metrics_utterances_level = compute_evaluation_utterances(dialog)
for key, value in metrics_utterances_level.items():
    print(f"{key}: {value}")

# Audio level evaluation
metrics_audio_level = compute_evaluation_audio(dialog)
for key, value in metrics_audio_level.items():
    print(f"{key}: {value}")

## Audio level evaluation

In [ ]:
# TODO: Evaluate the final generated audio